In [1]:
import os
import xarray as xr

path = rf"E:\client_v2_data"

files = os.listdir(path)
files = [f for f in files if f.endswith('.tif')]

In [31]:
WB_country = xr.open_dataset(rf"{path}\WB_country_grid.nc")

minx = WB_country.x.min()
maxx = WB_country.x.max()
miny = WB_country.y.min()
maxy = WB_country.y.max()

In [14]:
# Crop GPW according to the country grid
for file in files:
    (
        xr.open_dataarray(os.path.join(path, files[0]))
            .sel(x=slice(minx, maxx), y=slice(maxy, miny))
            .rio.to_raster(os.path.join(path, file.replace('.tif', '_proc.tif')))
    )
    print(f"Se creó {os.path.join(path, file.replace('.tif', '_proc.tif'))}")

In [ ]:
### Create a dataframe with the unique combinations of ADM1_CODE and ADM2_CODE
df = WB_country.to_dataframe().reset_index()

# Create a unique ID for each unique combination of ADM1_CODE and ADM2_CODE
df['unique_id'] = df.groupby(['ADM1_CODE', 'ADM2_CODE']).ngroup()
df.to_parquet(rf"{path}\df_country_grid.parquet")

In [24]:
### Rebuild the xr.DataSet from the parquet file
from tqdm import tqdm
import pandas as pd
import numpy as np

# df = pd.read_parquet(rf"{path}\df_country_grid.parquet")
y_values = df.y.unique()
y_values = np.array_split(y_values, 100)
das = []
for y_chunk in tqdm(y_values):
    df_y = df[df.y.isin(y_chunk)]
    df_y = df_y.drop(columns=["ADM1_CODE", "ADM2_CODE"]).set_index(['y', 'x']).fillna(0).astype(np.uint16)
    df_y.to_xarray()["unique_id"].to_netcdf(rf"{path}\chunks\df_country_grid_{y_chunk[0]}.nc")

100%|██████████| 100/100 [21:51<00:00, 13.11s/it]


In [2]:
from dask.diagnostics import ProgressBar
chunks = os.listdir(r"E:\client_v2_data\chunks")
chunks = [f for f in chunks if f.endswith('.nc')]

das = []
for chunk in chunks:
    das.append(xr.open_dataarray(os.path.join(r"E:\client_v2_data\chunks", chunk), chunks={"x": 10000}))

with ProgressBar():
    ds = xr.concat(das, dim="y")
    # ds = ds.sortby("y", ascending=False)
    ds.to_netcdf(rf"{path}\df_country_grid_compiled.nc")

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


[########################################] | 100% Completed | 27.86 s


In [4]:
ds = xr.open_dataset(rf"{path}\df_country_grid_compiled.nc")
ds = ds.sortby("y", ascending=False)
ds.to_netcdf(rf"{path}\df_country_grid_proc.nc")

In [ ]:
WB_country.drop_vars(["str_to_hash", "ADM1_CODE", "ADM2_CODE"])#.to_netcdf(rf"{path}\WB_country_grid_proc.nc")